In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2024-04-30 15:18:22.118560: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 15:18:22.831794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2024-04-30 15:18:23.366714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 15:18:23.401314: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 15:18:23.401525: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
%load_ext tensorboard

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation, RandomContrast, RandomBrightness, RandomTranslation
from pathlib import Path
import matplotlib.pyplot as plt
import pydot
from tensorboard.plugins.hparams import api as hp

In [4]:
# Set your data directory
data_dir = Path('leaf/').with_suffix('')

# Parameters
batch_size = 32
image_size = (150, 150)
num_classes = 15
epochs = 20

In [5]:
plant_labels = [
    "Ulmus carpinifolia",
    "Acer",
    "Salix aurita",
    "Quercus",
    "Alnus incana",
    "Betula pubescens",
    "Salix alba 'Sericea'",
    "Populus tremula",
    "Ulmus glabra",
    "Sorbus aucuparia",
    "Salix sinerea",
    "Populus",
    "Tilia",
    "Sorbus intermedia",
    "Fagus silvatica"
]
plant_labels = sorted(plant_labels)

In [6]:
image_count = len(list(data_dir.glob('*/*.tif')))
print(image_count)

1125


In [7]:
# Data preprocessing and augmentation
# newer keras.preprocessing.image_dataset_from_firectory does not support .tif files


datagen = ImageDataGenerator(
    rescale=1./255, # rescaled and resized to 256x256 here
    validation_split=0.2
)  # 20% validation split

# Load and preprocess data
train_generator = datagen.flow_from_directory(
    data_dir,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42,
    target_size=image_size
    )

validation_generator = datagen.flow_from_directory(
    data_dir,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42,
    target_size=image_size
    )

# test_generator = datagen.flow_from_directory(
#     data_dir,
#     batch_size=batch_size,
#     class_mode='categorical',
#     seed=42,
#     )

inv_map = {v: k for k, v in train_generator.class_indices.items()}

Found 900 images belonging to 15 classes.
Found 225 images belonging to 15 classes.


In [8]:
# images, labels = next(datagen.flow_from_directory(
#     data_dir,
#     batch_size=batch_size,
#     class_mode='categorical',
#     seed=42,
#     ))
# print(images.dtype, images.shape)
# print(labels.dtype, labels.shape)


In [9]:
# ds = tf.data.Dataset.from_generator(
#     lambda: datagen.flow_from_directory(data_dir, batch_size=batch_size, class_mode='categorical', seed=42),
#     output_types=(tf.float32, tf.float32),
#     output_shapes=([32,256,256,3], [32,15])
# )

train_generator = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([32,150,150,3], [32,15])
)

test_generator = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([32,150,150,3], [32,15])
)

2024-04-30 15:18:23.667319: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 15:18:23.667730: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 15:18:23.668069: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

In [11]:
train_generator = train_generator.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_generator = test_generator.cache().prefetch(buffer_size=AUTOTUNE)


In [12]:
# for image_batch, labels_batch in train_generator:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break


In [13]:
data_augmentation = tf.keras.Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.3),
  # RandomContrast(0.3)
  # # RandomBrightness(0.3),
  # RandomTranslation(0.3,0.3)
])


In [14]:
train_generator = train_generator.map(lambda x,y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE)

In [15]:

# Model creation
model = Sequential([
    # data_augmentation,
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    # Dense(256, activation='relu'),
    # Dense(128, activation='relu'),
    # Dense(64, activation='relu'),
    Flatten(),
    Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())
# Model training
history = model.fit(train_generator, epochs=epochs, validation_data=test_generator)

/home/lucypher/Desktop/forestry/.venv/lib64/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │       188,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 576,591 (2.20 MB)

 Trainable params: 576,591 (2.20 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20


: 

: 

In [ ]:
model.save('model_7.keras')

In [ ]:
dot_img_file = 'model_7.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
def preprocess_image(img_path, target_size=(150,150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

In [ ]:
test_image = preprocess_image('hahaha.jpg')
model.predict(test_image)


In [ ]:
idx = np.argmax(model.predict(test_image))
inv_map[idx]